In [2]:
%matplotlib inline
import warnings
from gwpy.time import to_gps, from_gps, tconvert
from astropy.time import Time
from gwpy.timeseries import TimeSeries
from gwtrigfind import find_trigger_files
from gwpy.table import EventTable

In [3]:
tstart = Time('2020-01-26 07:10:00', format='iso', scale='utc')
tend = Time('2020-01-26 08:10:00', format='iso', scale='utc')

filedate = '200126_0710_0810' #YYMMDD_start_end
filelocation = 'scatter_data/'

channels = ['ETMX_X','ETMX_Y','ETMX_Z','ETMY_X','ETMY_Y','ETMY_Z','ITMX_X','ITMX_Y','ITMX_Z']

In [4]:
%%time
#grab seismometer data
for ch in channels:
    data = TimeSeries.fetch('L1:ISI-GND_STS_'+ ch + '_DQ', tstart, tend, verbose=True)#, allow_tape=True)
    data.write(filelocation + 'L1seis' + ch + '_' + filedate + '.txt')

Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
Opening new connection to nds... connecte

In [5]:
%%time
gwdata = TimeSeries.fetch('L1:GDS-CALIB_STRAIN', tstart.gps, tend.gps, verbose=True)
gwdata.write(filelocation + 'L1calibstrain_' + filedate + '.txt')

Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='True'
Checking channels list against NDS2 database... Opening new connection to nds.ligo.caltech.edu... Opening new connection to nds.ligo-la.caltech.edu... 

/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py39/lib/python3.9/site-packages/gwpy/timeseries/core.py:1142: NDSWarning: unique NDS2 channel match not found for 'L1:GDS-CALIB_STRAIN.clean'
  warnings.warn(error.split('\n', 1)[0],
Error in write(): Connection refused
Error in write(): Connection refused
/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py39/lib/python3.9/site-packages/gwpy/timeseries/core.py:1142: NDSWarning: Failed to establish a connection[INFO: Error occurred trying to write to socket]
  warnings.warn(error.split('\n', 1)[0],


connected
    [nds.ligo-la.caltech.edu] set ALLOW_DATA_ON_TAPE='True'
Checking channels list against NDS2 database... 

/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py39/lib/python3.9/site-packages/gwpy/timeseries/core.py:1142: NDSWarning: unique NDS2 channel match not found for 'L1:GDS-CALIB_STRAIN.clean'
  warnings.warn(error.split('\n', 1)[0],


RuntimeError: Cannot find all relevant data on any known server.

In [7]:
gwdata = gwdata.resample(2048)
gwdata = gwdata.resample(512)
gwdata.write(filelocation + 'L1calibstrain_' + filedate + '_512Hz.txt')

In [8]:
#downsample seismometer data to 64Hz
for ch in channels:
    data = TimeSeries.read(filelocation + 'L1seis' + ch + '_' + filedate + '.txt')
    data = data.resample(64)
    data.write(filelocation + 'L1seis' + ch + '_' + filedate + '_64Hz.txt')

In [13]:
#rerun for freq>1 instead of freq>8
cache = find_trigger_files('L1:GDS_CALIB_STRAIN','omicron',tstart,tend,ext='h5')
trigtab = EventTable.read(cache,format = 'hdf5', path = 'triggers').filter('frequency>1','frequency<2048','snr<200')

EventTable.write(trigtab,filelocation + 'L1calibtriggers' + filedate + '.csv',format='csv') 

In [10]:
%%time
#download suspension data
data = TimeSeries.fetch('L1:SUS-ETMX_L2_WIT_L_DQ', tstart, tend, verbose=True)
data.write(filelocation + 'L1sus_ETMX_DQ_' + filedate + '.txt')

Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
CPU times: user 49.4 ms, sys: 14 ms, total: 63.4 ms
Wall time: 1.28 s


In [11]:
%%time
#download transmission data #alignment sensing control
data = TimeSeries.fetch('L1:ASC-X_TR_A_NSUM_OUT_DQ', tstart, tend, verbose=True)
data.write(filelocation + 'L1asc_trA_ETMX_DQ_' + filedate + '.txt')

Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
CPU times: user 335 ms, sys: 11.9 ms, total: 346 ms
Wall time: 1.82 s


In [12]:
%%time
#download data for push onto penultimate mass
data = TimeSeries.fetch('L1:SUS-ETMX_L2_DRIVEALIGN_L_OUT_DQ', tstart, tend, verbose=True)
data.write(filelocation + 'L1sus_drive_ETMX_DQ_' + filedate + '.txt')

Opening new connection to nds... connected
    [nds] set ALLOW_DATA_ON_TAPE='False'
Checking channels list against NDS2 database... done
CPU times: user 346 ms, sys: 11.3 ms, total: 357 ms
Wall time: 1.67 s
